In [ ]:
# Импортируем все необходимые библиотеки и задаем сид для рандомизатора
import pandas as pd
import numpy as np
import string
import pickle
#from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import recall_score, precision_recall_curve, confusion_matrix, accuracy_score, classification_report, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt_tab')

from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# Загрузка данных
data = pd.read_csv('data/482reviews.csv')
data.shape

: 

In [ ]:
data.head(5)

,review,assessment
0,"+красивая, качественная печать, качественые де...",1
1,3 Д ручки очень интересное увлечение для детей...,0
2,"4 года - буквы вообще учить не хочет, ничем ин...",1
3,"Азул — это та самая игрулька, куда мозги можно...",1
4,"Бочонки маленькие, фишки обычный картон. Не по...",0


: 

In [ ]:
# Количество позитивных (1) и отрицательных отзывов (0)
data['assessment'].value_counts()

assessment
1    268
0    214
Name: count, dtype: int64

: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['assessment'], test_size = 0.25, random_state = 1)
y_train.value_counts()

assessment
1    191
0    170
Name: count, dtype: int64

: 

In [ ]:
y_test.value_counts()

assessment
1    77
0    44
Name: count, dtype: int64

: 

In [ ]:
#Предобработка текста
snowball = SnowballStemmer(language = "russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language = "russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

: 

In [ ]:
# Создаем словарь с наиболее часто встречаемыми словами
processed = data["review"]
processed = processed.apply(lambda x: " ".join(tokenize_sentence(x,  remove_stop_words = True)))
processed

0      +красив качествен печа качествен дета +нрав ре...
1      3 д ручк очен интересн увлечен дет взросл поку...
2      4 год букв вообщ уч хочет нич интерес вызва уж...
3      азул — эт та сам игрульк мозг хорошеньк размя ...
4      бочонк маленьк фишк обычн картон не понрав пок...
                             ...                        
477    эт втор игр фирм нрав устраива игра интересн з...
478                                       якубович хвата
479                           ярк цвет больш обь упакова
480                          ярк плотн качествен детальк
481                       good at 3- 6 old the liked ит🧱
Name: review, Length: 482, dtype: object

: 

In [ ]:
all_words = []
for text in processed:
    words = word_tokenize(text)
    for w in words:
        all_words.append(w)

all_words = nltk.FreqDist(all_words)

# Print the result
print("Number of words: {}".format(len(all_words)))
print("Most common words: {}".format(all_words.most_common(15)))
word_features = [x[0] for x in all_words.most_common(2000)]

Number of words: 3423
Most common words: [('игр', 473), ('игра', 180), ('.', 179), ('очен', 176), ('эт', 142), ('``', 137), ('сам', 129), ('интересн', 103), ('ребенк', 101), ('прост', 90), ('качеств', 89), ('так', 88), ('лет', 81), ('куп', 78), ('дет', 77)]


: 

In [ ]:
# Функция для нахождения фич в тексте
def find_features(text):
    words = word_tokenize(text)
    features = {}
    for word in word_features:
        features[word] = word in words

    return features

: 

In [ ]:
# Обучение модели логистической регрессии
vectorizer = TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x,  remove_stop_words = True), token_pattern=None)
features = vectorizer.fit_transform(X_train)
logreg_model = LogisticRegression(random_state = 0)
logreg_model.fit(features, y_train)

LogisticRegression(random_state=0)

: 

In [ ]:
X = vectorizer.fit_transform(X_train)
y_pred = logreg_model.predict(X)

: 

In [ ]:
# Проверка правильности модели на конкретном примере
logreg_model.predict(features[40])

array([0], dtype=int64)

: 

In [ ]:
X_train.iloc[40]

'Ужасное качество исполнения. Отвратительное. Нельзя использовать здесь ненормативную лексику, а ой как хочется. Скобы степлера, которыми крепятся "сетки" в лузах частично пристреляны мимо. Борта, сделанные из ДСП, местами в сколах, хотя это не влияет на игру, конечно. Сукно на бортах плохо натянуто, потому ждать какого-то хорошего отскока шаров от них не придётся. Посреди самого стола, в самом центре, огромное пятно почти с половину стола, очень хотелось бы думать, что это клей, которых сукно приклеили к фанере стола. Короче, поставил бы одну звезду, но подарок ребёнку всё равно нравится, играют со старшим братом. Дороже 500 рублей бы не купил. Да вообще бы не купил, если бы жена проверила на качтство при получении., '

: 

In [ ]:
logreg_model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words=True), token_pattern=None)),
    ("model", LogisticRegression(random_state = 0 ))])
logreg_model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function <lambda> at 0x000001DD1E4ACFE0>)),
                ('model', LogisticRegression(random_state=0))])

: 

In [ ]:
# Получаем метрики точности
y_pred = logreg_model_pipeline.predict(X_test)
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

predicted         
                 negative positive
actual negative        35        9
       positive         5       72

: 

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8842975206611571
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.80      0.83        44
           1       0.89      0.94      0.91        77

    accuracy                           0.88       121
   macro avg       0.88      0.87      0.87       121
weighted avg       0.88      0.88      0.88       121



: 

In [ ]:
# Сохранение модели со словарем фич и функцие нахождения их в тексте
path = 'models\\naive_bayes_classifier.pickle'

with open(path, 'wb') as classifier_file:
    data_for_save = {
        'model': logreg_model,
        'features': word_features,
        'function': find_features,
        }
    pickle.dump(data_for_save, classifier_file)

: 

In [ ]:
#f = open('models\\naive_bayes_classifier.pickle', 'rb')
#sd = pickle.load(f)
#print(sd)

: 

In [ ]:
# Обучение байесовского классификатора
mulnb_model = MultinomialNB()
mulnb_model.fit(features, y_train)
mulnb_model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words=True), token_pattern=None)),
    ("model", MultinomialNB())])

: 

In [ ]:
mulnb_model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function <lambda> at 0x000001DD209585E0>)),
                ('model', MultinomialNB())])

: 

In [ ]:
y_pred_B = mulnb_model_pipeline.predict(X_test)
pd.DataFrame(
    confusion_matrix(y_test, y_pred_B),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

predicted         
                 negative positive
actual negative        33       11
       positive         8       69

: 

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8429752066115702
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.75      0.78        44
           1       0.86      0.90      0.88        77

    accuracy                           0.84       121
   macro avg       0.83      0.82      0.83       121
weighted avg       0.84      0.84      0.84       121



: 

In [ ]:
# Сохранение модели со словарем фич и функцие нахождения их в тексте
path = 'models\\logistic_regression_classifier.pickle'

with open(path, 'wb') as classifier_file:
    data_for_save = {
        'model': mulnb_model,
        'features': word_features,
        'function': find_features,
        }
    pickle.dump(data_for_save, classifier_file)

: 

In [ ]:
# Если мы хотим найти 95% негативных комментариев, то...
precision_score(y_test, y_pred)

0.8625

: 

In [ ]:
recall_score(y_test, y_pred)

0.8961038961038961

: 

In [ ]:
prec, rec, thresholds = precision_recall_curve(y_test, probas_pred=model_pipeline.predict_proba(X_test)[:, 1])

D:\DataS\Lib\site-packages\sklearn\metrics\_ranking.py:993: FutureWarning: probas_pred was deprecated in version 1.5 and will be removed in 1.7.Please use ``y_score`` instead.
  warnings.warn(


: 

In [ ]:
np.where(prec > 0.95)

(array([ 53,  54,  55,  56,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
         83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
         96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
        109, 110, 111, 112, 113, 114, 115, 116], dtype=int64),)

: 

In [ ]:
thresholds[53]

0.584075708207464

: 

In [ ]:
pd.DataFrame(
    confusion_matrix(y_test, model_pipeline.predict_proba(X_test)[:, 1] > thresholds[36]),
    index=[["actual", "actual"], ["negative", "positive"]],
    columns=[["predicted", "predicted"], ["negative", "positive"]],
)

predicted         
                 negative positive
actual negative        35        9
       positive         5       72

: 

In [ ]:
#Мы можем найти все 44 негативных комментария из 44 при thresholds[78]! thresholds[8] - Найдем все позитивные комментарии.
#При thresholds[36] наибольшая точность в 88%

: 